In [ ]:
import pandas as pd
df1 = pd.read_csv(r'C:\Users\mtsil\OneDrive\Documents\finalcorpus.csv', ';', encoding= 'unicode_escape') #call newdf as df1
df1.head()

In [ ]:
x = df1.drop('label', axis=1)
#dependent feature 
y = df1['label']

In [ ]:
vocab_size= 20000
from tensorflow.keras.preprocessing.text import one_hot
encoding = [one_hot(d,vocab_size) for d in df1['text']]

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
emb_doc = pad_sequences(encoding,padding='pre',maxlen=20)

In [ ]:
import numpy as np
final_x = np.array(emb_doc)
final_y = np.array(y)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional

model = Sequential()
model.add(Embedding(vocab_size, output_dim=50, input_length=20))
model.add(Bidirectional(LSTM(4)))
model.add(Dense(2,activation='relu'))

model.add(Dropout(0.5)) # either add a dropout layer or a new Dense layer
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer= 'adam', metrics=['accuracy'])
print(model.summary())

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(final_x, final_y, test_size=0.20, random_state=42)
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), shuffle =  True, epochs=10, batch_size=64)

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
import matplotlib.pyplot as plt
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
plt.plot(epochs, acc, 'y', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
import pandas as pd

history.history

history_df = pd.DataFrame(list(zip(history.history['loss'],history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy'])),
                          columns=['loss','accuracy','val_loss','val_accurary'])
history_df['epoch']=list(range(1,len(history_df['loss'])+1,1))
history_df